In [ ]:
__author__ = 'Robert Nikutta <robert.nikutta@noirlab.edu>, Stéphanie Juneau <stephanie.juneau@noirlab.edu>, Knut Olsen <knut.olsen@noirlab.edu>, David Nidever <david.nidever@noirlab.edu>, Alice Jacques <alice.jacques@noirlab.edu>'
__version__ = '20231029' # yyymmdd; version stamp of this notebook
__datasets__ = ['smash_dr1', 'delve_dr2'] # datasets used in this notebook
__keywords__ = ['science example','dwarf galaxies'], # keywords relevant to this notebook, e.g. ['science case','dwarf galaxies'] 

# Detecting the Hydra II dwarf galaxy in SMASH DR1 - with exercises

*Robert Nikutta, Stéphanie Juneau, Knut Olsen, David Nidever, Alice Jacques & Astro Data Lab Team*

### Table of contents
* [Goals & notebook summary](#goals)
* [Disclaimer & Attribution](#attribution)
* [Imports & setup](#import)
* [Query the SMASH catalog for blue stars (expected in metal-poor dwarfs)](#query)
* [Spatial density plot](#spatial)
* [Detect overdensities (differential Gaussian kernels)](#kernel)
* [Find peaks](#peaks)
* [Visually inspect images (Simple Image Access)](#sia)
* [CM diagrams of all stars around the peaks](#cmdiagrams)
* [Save your results to a local file](#savefile)
* [Exercise: Detect the Centaurus I dwarf galaxy in DELVE DR2](#exercise)
* [Resources](#resource)


<a class="anchor" id="goals"></a>
# Goals

We will use a dwarf galaxy detection algorithm using a spatial overdensity of blue stars to re-discover the Hydra II dwarf galaxy in the SMASH DR1 dataset.  Hydra II (red circle in the map below) is one of ~60 dwarf galaxy satellites of the Milky Way.

<img style="float: left;" src="dwarfmap_nipy_spectral_r.png">


# Summary

**Background**
Ultrafaint dwarf galaxies are crucial to understanding many aspects of the universe. For instance, they are dominated by dark matter; their localization in space can thus trace the large-scale structure of the dark matter distribution. Furthermore, dwarf galaxies are suspected to host intermediate-mass black holes (IMBH), which so far have eluded efforts to find them. IMBHs will naturally bridge the gap between the solar-mass black hole and super-massive blackholes that reside at the center of virtually every large galaxy.

**Data retrieval**
We will retrieve data from Field 169 of the SMASH catalog ([Nidever et al. (2017, AJ, 154, 199)](http://adsabs.harvard.edu/abs/2017AJ....154..199N)) and look for overdensities of blue objects.

The required columns are RA, Dec, and the g-band and r-band magnitudes.

**Detection**
We will convolve the spatial distribution of our dataset with a pair of Gaussian kernels and subtract the results, as done in e.g. [Stanford et al. (2005, ApJ, 634, 2, L129)](http://adsabs.harvard.edu/abs/2005ApJ...634L.129S) (galaxy clusters), or [Koposov et al. (2008, ApJ, 686, 279)](http://adsabs.harvard.edu/abs/2008ApJ...686..279K) (MW satellites). This has the effect of convolving the spatial distribution with a Mexican hat filter, which is useful for detecting objects at a desired spatial scale.

<a class="anchor" id="attribution"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

# Imports and setup

In [ ]:
# std lib
from getpass import getpass
import warnings
warnings.filterwarnings('ignore') # to suppress some astropy depracation warnings

# 3rd party
import numpy as np
from astropy import utils, io, convolution, stats
from astropy.visualization import make_lupton_rgb
from photutils import find_peaks
from pyvo.dal import sia
import pylab as plt
%matplotlib inline

# Data Lab
from dl import authClient as ac, queryClient as qc

# plots default setup
plt.rcParams['font.size'] = 14

 <a class="anchor" id="query"></a>
# Query the SMASH DR1 database

We will query the averaged photometry table from the SMASH catalog and select field #169.  We'll issue two queries, one in which we select blue (-0.4 < g-r < 0.4) star-like objects (|sharp|<0.5), and the other without those constraints, to be used for comparison. For both queries, we will avoid photometry taken only with short exposures (depthflag>1), exclude objects with fewer than 4 detections, and apply a magnitude constraint of 9 < g < 25, so as to avoid excessive contamination.

## Construct the query strings

In [ ]:
field = 169 # SMASH field number to query
# Create a query of all sources in the Field, for comparison
query_all =\
"""SELECT ra,dec,gmag,rmag
   FROM smash_dr1.object
   WHERE fieldid = '{:d}' AND
         depthflag > 1 AND
         ndetr > 3 AND ndetg > 3 AND
         gmag BETWEEN 9 AND 25 AND random_id<10""".format(field)

In [ ]:

# Create the query string; SQL keyword capitalized for clarity
#   depth > 1 = no short exposures please
#   ndetr, ndetg > 3 = more than 3 detections in r & g bands
#   abs(sharp) < 0.5 = avoid broad objects
query =\
"""SELECT ra,dec,gmag,rmag
   FROM smash_dr1.object
   WHERE fieldid = '{:d}' AND
         depthflag > 1 AND
         ndetr > 3 AND ndetg > 3 AND
         abs(sharp) < 0.5 AND
         gmag BETWEEN 9 AND 25 AND
         (gmag-rmag) BETWEEN -0.4 AND 0.4""".format(field)

# Submit the queries
Running the query in synchroneous mode is very easy.

In [ ]:
df_all = qc.query(sql=query_all,fmt='pandas') # response is by default a CSV-formatted string
print("Number of objects:", len(df_all))
print(df_all.head())

In [ ]:
R = qc.query(sql=query,fmt='pandas') # response is by default a CSV-formatted string
print("Number of objects:", len(R))
print(R.head())

<a class="anchor" id="spatial"></a>
# Make a figure of the spatial distribution

You might spot some overdensities already, particularly in the result with blue star-like objects.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(14,7), sharey=True, constrained_layout=True)
h1 = ax1.hexbin(df_all['ra'], df_all['dec'],gridsize=200)
ax1.set_xlabel('RA [deg]')
ax1.set_ylabel('Dec [deg]')
ax1.set_title('{:d} random objects'.format(len(df_all)))
h2 = ax2.hexbin(R['ra'], R['dec'],gridsize=200)
ax2.set_xlabel('RA [deg]')
ax2.set_title('{:d} blue stellar objects'.format(len(R)))
fig.colorbar(h1,label='number of objects per spatial bin');

<a class="anchor" id="kernel"></a>
# The Dwarf Filter
Here we define the dwarf filter as a differential convolution of a two-dimensional image using two Gaussian kernels; this has the effect of convolution with a Mexican hat filter.  The default kernel shapes look for objects on the scale of a few arcmin.  The output includes a clipped array of the convolved spatial distribution, which we will use for peak detection.

In [ ]:
def dwarf_filter (ra,dec,fwhm_small=2.0,fwhm_big=20):

    """Differential convolution with 2D Gaussian kernels.
    
       Based on Koposov et al. (2008).
       Code by Ken Mighell and Mike Fitzpatrick.
       Minor edits by RN.
       
       Parameters
       ----------
       ra, dec : float or array
           RA & Dec in degrees.
    
       fwhm_small, fwhm_big : float
           Full-width half maximum sizes of the small and big Gaussian kernels
           to use in convolution, in arcminutes.
    """
    
    x, y = ra, dec

    print("Computing differential convolution .... ",)

    # Information about declination (y) [degrees]
    ymean = (y.min() + y.max()) / 2.0
    ydiff_arcmin = (y.max() - y.min()) * 60.0 # convert from degrees to arcmin

    # Information about right ascension (x) [degrees in time]:
    xdiff = x.max() - x.min() # angular separation [degrees (time)] 
    xmean = (x.min() + x.max()) / 2.0

    # convert from degrees in time to separation in angular degrees:
    xdiff_angular = (x.max() - x.min()) * np.cos(ymean*(np.pi/180.0))

    # convert from degress to arcmin
    xdiff_angular_arcmin = xdiff_angular * 60.0 

    # Get the number of one-arcmin pixels in the X and Y directions:
    nx = np.rint(xdiff_angular_arcmin).astype('int')
    ny = np.rint(ydiff_arcmin).astype('int')

    # Create a two-dimensional histogram of the raw counts:
    Counts, xedges, yedges  = np.histogram2d (x, y, (nx,ny) )
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    raw_hist = Counts.T.copy()

    # Make the small and big Gaussian kernels with a standard deviation
    # of the given FWHM in arcmin^2 pixels.
    kernel_small = convolution.Gaussian2DKernel(fwhm_small/2.35,factor=1)
    kernel_big = convolution.Gaussian2DKernel(fwhm_big/2.35,factor=1)

    # Compute the differential convolution kernels.
    conv_big = convolution.convolve(raw_hist, kernel_big)
    conv_small = convolution.convolve(raw_hist, kernel_small)
    conv_delta = conv_small - conv_big
    delta = conv_delta.copy()

    # Compute statistics and the floor
    mean = np.mean(delta, dtype='float64')
    sigma = np.std(delta, dtype='float64')
    sigmaRaw = np.std(raw_hist,dtype='float64')
    floor = mean

    print('dwarf_filter: mean = {:g}  sigma = {:g} sigmaRaw = {:g}'.format(mean, sigma, sigmaRaw))

    clipped = delta.copy()
    clipped[delta < floor] = floor

    # Return the computed fields.
    return raw_hist, extent, delta, clipped, sigma

# Run the dwarf filter
We'll use the default convolution kernels of 2 and 20 arcminutes in size.

In [ ]:
small_k, big_k = 2.0, 20.  # kernel sizes in arcminutes
raw, extent, delta, clipped, dsigma = dwarf_filter(R['ra'],R['dec'],fwhm_small=small_k,fwhm_big=big_k)

# Plot the convolved 2D histogram

In [ ]:
fig, ax = plt.subplots(figsize=(8,7))
im = plt.imshow(clipped, origin='lower', aspect='auto')
plt.xlabel('Pixel')
plt.ylabel('Pixel')
plt.colorbar(label='relative spatial density after convolution');

**Some peaks are visible, let's locate them automatically...**

<a class="anchor" id="peaks"></a>
# Identify spatial density peaks
We'll use the `photutils` package to identify 10-sigma peaks in the clipped filtered image.

In [ ]:
# find peaks
mean, median, std = stats.sigma_clipped_stats(clipped,sigma=3.0,maxiters=5)    
tbl = find_peaks(clipped,median+10,box_size=small_k*2)

# add ra & dec positions of peaks found
start, stop = extent[:2]
step_size = (stop-start)/clipped.shape[1]
xvec = np.arange(start,stop+step_size,step_size)
start, stop = extent[2:]
step_size = (stop-start)/clipped.shape[0]
yvec = np.arange(start,stop+step_size,step_size)

tbl['ra'] = xvec[tbl['x_peak']]
tbl['dec'] = yvec[tbl['y_peak']]
print(tbl)

# Show the identified density peaks

In [ ]:
ecs = ['y','w'] # colors of box frames
ax.scatter(tbl['x_peak'],tbl['y_peak'],marker='s',s=tbl['peak_value']*40,c='none',edgecolors=ecs,lw=3)
fig

<a class="anchor" id="sia"></a>
# Inspect the image cutouts around the peaks
# *Simple Image Access service*
Data Lab comes with batteries included. Image cutout and download services are built in.

We'll just write two little functions:

* one to download the deepest stacked images found in the given bands at a given position in the sky
* and a function to plot several images side-by-side.

In [ ]:
# set up SIA
DEF_ACCESS_URL = "https://datalab.noirlab.edu/sia/coadd_all"
svc = sia.SIAService(DEF_ACCESS_URL)

# to download the deepest stacked images
def download_deepest_images(ra,dec,fov=0.1,bands=list('gri')):
    imgTable = svc.search((ra,dec), (fov/np.cos(dec*np.pi/180), fov), verbosity=2).to_table()
    print("The full image list contains {:d} entries.".format(len(imgTable)))
    sel0 = (imgTable['proctype'] == 'Stack') & (imgTable['prodtype'] == 'image') # basic selection
    images = []
    for band in bands:
        print("Band {:s}: ".format(band)) #, end='')
        sel = sel0 & (imgTable['obs_bandpass'] == band) # add 'band' to selection
        Table = imgTable[sel] # select
        row = Table[np.argmax(Table['exptime'].data.data.astype('float'))] # pick image with longest exposure time
        url = row['access_url'] # get the download URL
        print('downloading deepest stacked image...')
        img = io.fits.getdata(utils.data.download_file(url,cache=True,show_progress=False,timeout=120)) # .decode() b/c in Python 3 url is of "byte" type and getdata() expects "string" type
        images.append(img)
        
    print("Downloaded {:d} images.".format(len(images)))
    return images

# to make multi panel image
def plot_images(images,titles=list('gri'),cmap=plt.cm.gray_r):
    fig = plt.figure(figsize=(8,10),tight_layout=True)
    grid = plt.GridSpec(4, 3)
    ax1 = fig.add_subplot(grid[0, 0])
    ax2 = fig.add_subplot(grid[0, 1])
    ax3 = fig.add_subplot(grid[0, 2])
    ax4 = fig.add_subplot(grid[1:, 0:])

    for j,ax,img in zip(np.arange(4),[ax1,ax2,ax3,ax4],images):
        ax.imshow(img,origin='lower',interpolation='none',cmap=cmap,norm=plt.mpl.colors.PowerNorm(0.1))
        ax.axis('off')
        ax.set_title('{:s}'.format(titles[j]))
    

# Get images for the "left yellow" box
Download the deepest stacked image cutouts (in 3 bands) around the position of the peak marked with a yellow box. Depending on network speed and system load, this can take a few seconds. Also create a 3-band false-color composite.

In [ ]:
bands = list('gri')
idx = 0
print(tbl['ra'][idx], tbl['dec'][idx])
images = download_deepest_images(tbl['ra'][idx], tbl['dec'][idx], fov=0.1, bands=bands) # FOV in deg

Plot the images, plus a false-color 3-band image:

In [ ]:
images = [im-np.median(im) for im in images] # subtract median from all images for better scaling
images += [make_lupton_rgb(*images[::-1],stretch=30)] # add a 3-color composite image
plot_images(images,titles=bands+['3-band image'])

**Looks like a galaxy cluster!**

# Now get images for the "white center" box

In [ ]:
idx = 1
print(tbl['ra'][idx], tbl['dec'][idx])
images = download_deepest_images(tbl['ra'][idx], tbl['dec'][idx], fov=0.1, bands=bands) # FOV in deg
images = [im-np.median(im) for im in images] # subtract median from all images for better scaling
images += [make_lupton_rgb(*images[::-1],stretch=30)] # add a 3-color composite image
plot_images(images,titles=bands+['3-band image'])

**Definitely not a galaxy cluster, but a stellar overdensity**

<a class="anchor" id="cmdiagrams"></a>
# Query the database for stars near peaks
Get photometry of stars within a 5 arcmin radius of each density peak.

In [ ]:
def makequery(ra0,dec0,radius0=5./60.,field=169):
    query = """
SELECT ra,dec,gmag,rmag FROM smash_dr1.object
  WHERE fieldid = {:d}
    AND depthflag > 1
    AND abs(sharp) < 0.5
    AND gmag BETWEEN 9 AND 25
    AND q3c_radial_query(ra,dec,{:f},{:f},{:f})
""".format(field,ra0,dec0,radius0)

    return query

For more information on the q3c function used above, see the [q3c GitHub page](https://github.com/segasai/q3c).

First, retrieve the data for the "left yellow" box. 

In [ ]:
query0 = makequery(tbl['ra'][0],tbl['dec'][0]) # center ra & dec
R0 = qc.query(sql=query0, fmt='pandas') # using sql argument instead of the default adql
print(R0.head()) # a Pandas method

Then compute the g-r color and add it to as a new column to the table.

In [ ]:
R0['g_r'] = R0['gmag'] - R0['rmag']
print( R0.head())

Do the same for the "white center" box.

In [ ]:
query1 = makequery(tbl['ra'][1],tbl['dec'][1])
R1 = qc.query(sql=query1,fmt='pandas')
R1['g_r'] = R1['gmag'] - R1['rmag'] # compute color
print(R1.tail()) # another Pandas methodb

# Plot color-magnitude diagrams of the peaks

In [ ]:
def plotpanel(axid,x,y,title='',xlim=(-1,2),ylim=(25.2,14)):
    ax = fig.add_subplot(axid)
    ax.scatter(x,y,marker='.',s=10, alpha=0.8)
    ax.set_xlabel(x.name)
    ax.set_ylabel(y.name)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_title(title)

In [ ]:
fig = plt.figure(figsize=(12,5.5))
plotpanel(121,R0['g_r'],R0['gmag'],'yellow box, {:d} objects'.format(len(R0)))
plotpanel(122,R1['g_r'],R1['gmag'],'white box, {:d} objects'.format(len(R1)))

# Compare w/ Vivas+2016
￼￼￼<img style="float: left;" src="Vivas2016fig9.png">

# Save your photmetry table
If you wish, you can now save your photometry table for Hydra II to a local file and take it with you.

In [ ]:
outfile = 'hydra2.csv'
R0.to_csv(outfile,index=False)

<a class="anchor" id="exercise"></a>
# Exercise: Detect the Centaurus I dwarf galaxy in DELVE DR2
Now we will run through the same process to re-discover the Centaurus I dwarf galaxy in the DELVE DR2 dataset ([Mau et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...890..136M/abstract)).

### 1. Fill in the blanks in the query
Query the `delve_dr2.objects` table for the following columns:
- ra
- dec
- mag_auto_g
- mag_auto_r

And with the following constraints:
- Within a radius of 0.5 arcmins of RA=189.58 and Dec=-40.89
- Limit the g-band value between 14 and 23
- Only blue objects (-0.4 < g-r < 0.4)

In [ ]:
query = """
SELECT ra, dec, mag_auto_g, mag_auto_r
FROM __
WHERE q3c_radial_query(ra, dec, __, __, 0.5)
      AND __ BETWEEN __ AND __
      AND (__ - __) BETWEEN __ AND __"""
print(query)

### 2. Submit the query
Run the query in synchroneous mode and make the output format `pandas`.

In [ ]:
df = __
print("Number of objects:", len(df))
print(df.head())

### 3. Make a figure of the spatial distribution

In [ ]:
fig = plt.figure(figsize=(7,6))
plt.hexbin(__, __, gridsize=200)
plt.xlabel('RA [deg]')
plt.ylabel('Dec [deg]')
plt.colorbar(label='number of objects per spatial bin')

### 4. Run the dwarf filter using the same dwarf filter function from before
We'll use the default convolution kernels of 2 and 20 arcminutes in size.

In [ ]:
small_k, big_k = __, __  # kernel sizes in arcminutes
raw, extent, delta, clipped, dsigma = dwarf_filter(__, __, fwhm_small=__, fwhm_big=__)

### 5. Plot the convolved 2D histogram
One dense peak will be visible.

In [ ]:
fig, ax = plt.subplots(figsize=(7,6))
im = plt.imshow(__)
plt.xlabel('pixel')
plt.ylabel('pixel')
plt.colorbar(label='relative spatial density after convolution')

### 6. Identify peaks
We'll use the `photutils` package to identify 3-sigma peaks in the clipped filtered image.

In [ ]:
# find peaks
mean, median, std = stats.sigma_clipped_stats(__, sigma=3.0, maxiters=5)
print(f'Mean = {mean}, Median = {median}, Std = {std}')
tbl = find_peaks(__, median+3, box_size=small_k*2)

# add ra & dec positions of peaks found
start, stop = extent[:2]
step_size = (stop-start)/clipped.shape[1]
xvec = np.arange(start,stop+step_size,step_size)
start, stop = extent[2:]
step_size = (stop-start)/clipped.shape[0]
yvec = np.arange(start,stop+step_size,step_size)

tbl['ra'] = xvec[tbl['x_peak']]
tbl['dec'] = yvec[tbl['y_peak']]
print(tbl)

### 7. Show the identified density peak

In [ ]:
ecs = ['w'] # color of box frame
ax.scatter(__, __, marker='s', s=tbl['peak_value']*40,
           c='none', edgecolors=ecs, lw=3)
fig

### Additional ideas for advanced users

* Find overdensities elsewhere (everywhere!)

* Optimize kernel sizes automatically, computation, proper wavelets

# Resources

Koposov et al. (2008, ApJ, 686, 279) "The Luminosity Function of the Milky Way Satellites": http://adsabs.harvard.edu/abs/2008ApJ...686..279K

Martin et al. (2015, ApJ, 804, 5) "Hydra II: A Faint and Compact Milky Way Dwarf Galaxy Found in the Survey of the Magellanic Stellar History": http://adsabs.harvard.edu/abs/2015ApJ...804L...5M

Mau et al. (2020, ApJ, 890, 2 ) "Two Ultra-faint Milky Way Stellar Systems Discovered in Early Data from the DECam Local Volume Exploration Survey": https://ui.adsabs.harvard.edu/abs/2020ApJ...890..136M

Nidever et al. (2017) "SMASH - Survey of the MAgellanic Stellar History": http://adsabs.harvard.edu/abs/2017AJ....154..199N

Stanford et al. (2005, ApJ, 634, 2, L129) "An IR-selected Galaxy Cluster at z = 1.41": http://adsabs.harvard.edu/abs/2005ApJ...634L.129S